In [1]:
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

df = pd.read_csv("../input/diabets/diabetes.csv")

In [6]:
## 1. Exploratory Data Analysis

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df.shape

(768, 9)

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [9]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [14]:
## 2. Data Preprocessing & Feature Engineering

y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

#Önce verileri sıtandartlaştırmak için standartscaler kullandık.
#Böylece işlemlerimiz daha hızlı olacak.
X_scaled = StandardScaler().fit_transform(X)

#Yukarıdaki işlem sonucunda x değerleri array ile döndürüldü.
#Buna sütun eklemek ve dataframe e dönüştürmek için aşağıdaki işlemi yaptık.
X = pd.DataFrame(X_scaled, columns=X.columns)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


In [20]:
# 3. Modeling & Prediction

knn_model = KNeighborsClassifier().fit(X, y)

# x lerden rasgele 1 örneklem seç diyorum.
random_user = X.sample(1, random_state=45)

knn_model.predict(random_user)

array([1])

In [21]:
## 4. Model Evaluation - Model Başarı Değerlendirme

# Confusion matrix için y_pred:
#KNN kullanarak tüm gözlem birimleri için tahmin yap y_pred'e ata
#Bunu karmaşıklık matrisi hesaplamak için kullanacağız.
y_pred = knn_model.predict(X)


# AUC için y_prob:
#AUC yi hesağlatmak için 1'e ait olma olas getirdim ki eğri altında kalan alanı bbulayım.
y_prob = knn_model.predict_proba(X)[:, 1]

print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



Yukarıdaki sonuca göre accr. oranımız 0.83.Yani %83 modelimiz başarılı sonuç veriyor.Ancak accr. her zaman doğru sonuç vermeyebilir.Dengesiz bir verimiz varsa (bkz support 0=500,1=268) başka metriklere de bakmalıyız.Yani press, recall ve f1 score. f1 scorea baktığımda 0.74 çıkmış.

In [22]:
roc_auc_score(y, y_prob) # y ler ve y lerde yer alan 1 sınıfına ait olma olas.

0.9017686567164179

Modeli yine train set ile test ettik.Şimdi test verisi ile test edelim.Bunu cross validation ile yapalım.

In [24]:
#cross vald. der ki: bana model nesneni,bağımsız ve bağımlı değişkenini, 
#kaç katlı doğru yapacağını ve cross val.da kullanmak istediğin metrikleri ver

cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

{'fit_time': array([0.00336146, 0.00252748, 0.00245333, 0.00244689, 0.00237441]),
 'score_time': array([0.01337767, 0.01130128, 0.0115304 , 0.01114011, 0.01140857]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

In [25]:
# Her metrikte bu 5 kat için çıkan sonuçların ort. alıyoruz.

cv_results['test_accuracy'].mean()

0.733112638994992

In [26]:
cv_results['test_f1'].mean()

#Sonuca baktığımızda f1 score train sette olduğundan oldukça düşük. 
#Daha önceki veriye bakarak yorum yapsaydık 0.73 güzel bir rakamdı ve 
#verimiz dengesiz değil göz ardı edilebilir diyip ilerlemeyebilirdik.

0.5905780011534191

In [27]:
cv_results['test_roc_auc'].mean()

0.7805279524807827

Peki bu başarı scorelarını nasıl arttırabiliriz?

1. Örnek boyutu arttıralabilir.
2. Veri ön işleme işlemleri detaylandırılabilir.
3. Özellik mühendisliği  ( yeni değişkenler türetilebilir)
4. İlgili algoritma için optimizasyonlar yapılabilir. Örn komşuluk sayısı hiperparametrsi değiştirilebilir.

In [28]:
# knn model param. görelim

knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

**Parametre:** modellerin veri içinden öğrendiği ağırlıklardır.
**Hiperparametre :** Kullanıcı tarafından tanımlanması gereken,veri setinden öğrenilemeyen parametrelerdir.Örn kaç komşuluk için işlem yapılacak.Default değeri 5 ama biz bunu değiştirebiliriz.

In [33]:
## 5. Hyperparameter Optimization

knn_model = KNeighborsClassifier()
knn_model.get_params()

#Burada komşuluk sayısı 5 verilmiş.Biz olabilecek optimum komşuluk
#sayısını bulmak için farklı komşuuluk sayılarını vererek işlemler yaptıracağız.

knn_params = {"n_neighbors": range(2, 50)} #komşu sayısını 2 den 50 ye kadar girdim.


{'n_neighbors': range(2, 50)}

In [34]:
#GridSearchCV der ki bana modelini göster,hangi parametre setini deneyeceksin,
#kaç katlı cross vald. yapacaksın? burada hiperparm. opt. için kaç katlı bakmamız gerek onu veriyoruz.
#Yani 2,3,4,5,6 yı kendi arasında dene (5 sayıyı)
#n_jobs -1 işlemcileri tam kapast kullanır.
#verbose=1 rapor beklediğimi belirtiyorum.

knn_gs_best = GridSearchCV(knn_model,
                           knn_params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1).fit(X, y)

#sonuca göre 48 aday var.Yani 2 den 50 ye.Her biri için 5 katlı 
#yapılacağından 240 işlem var.


Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [35]:
#Bunların içinden optimum komşuluk sayısını bulmak için:

knn_gs_best.best_params_

#Sonuç 17 geldi.yani ben artık komşuluğu 17 vererek bir final modeli bunun üzerinden kurmalıyız.

{'n_neighbors': 17}

In [37]:
## 6. Final Model
#bunu 2 yıldızla yazdım çünkü normalde birden fazla parametre için bu işlemleri yapmam gerekebilirdi.
#2 yıldız olunca bulduğum değerleri buraya uygula demiş oldum.
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)

cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [38]:
cv_results['test_accuracy'].mean()
#accr score artmış

0.7669892199303965

In [39]:
cv_results['test_f1'].mean()
#f1 score artmış

0.6170909049720137

In [40]:
cv_results['test_roc_auc'].mean()
#auc score artmış

0.8127938504542278

In [49]:
random_user = X.sample(1)
knn_final.predict(random_user)

array([0])